In [1]:
import pathlib
import pandas as pd
import json
import re

In [ ]:
path_test2_swt = pathlib.Path(r"C:\Users\djock\JEDHA Full_Stack\JEDHA_FullStack_Projet AURA\data_AURA\feats-v0_4\train\01_tcp_ar\119\00011972\s001_2014_09_16\00011972_s001_t000-xqrs.json")


path_test3_xqrs = pathlib.Path(r"C:\Users\djock\JEDHA Full_Stack\JEDHA_FullStack_Projet AURA\data_AURA\\res-v0_4\train\01_tcp_ar\119\00011972\s001_2014_09_16\00011972_s001_t000.json")


#path_test2_swt_annot = pathlib.Path("C:\\Users\\djock\\JEDHA Full_Stack\\JEDHA_FullStack_Projet AURA\\data_AURA\\annot-v0_4\\train\\01_tcp_ar\\002\\00000254\\s007_2013_03_25\\00000254_s007_t000.json")
#path_test3_pan = pathlib.Path("C:\\Users\\djock\\JEDHA Full_Stack\\JEDHA_FullStack_Projet AURA\\data_AURA\\feats-v0_4\\train\\01_tcp_ar\\118\\00011869\\s005_2014_08_13\\00011869_s005_t005-pan.json")

## load des fichiers JSON

In [68]:
data1 = json.load(open(path_test2_swt, "r"))
data2 = json.load(open(path_test3_xqrs, "r"))
#data_annot1 = json.load(open(path_test2_swt_annot, "r"))

In [69]:
df1 = pd.DataFrame(data=data1["features"], columns=data1["keys"])
df2 = pd.DataFrame(data=data2["infos"], columns=data2["keys"])
#df3_annot = pd.DataFrame(data=data_annot1["features"], columns=data_annot1["keys"])

KeyError: 'features'

In [11]:
def load_and_concat_feats(dir, type):
    '''
    Directory : The path of the feats-v0_4 folder,
                could be relative or absolute path
    Type : Select which type of file to handle (pan, swt, xqrs)
    '''
    full_df = pd.DataFrame()
    for path in pathlib.Path(dir).rglob(f'*{type}.json'):
        data = json.load(open(path, "r"))
        df = pd.DataFrame(columns = data['keys'], data = data['features'])
        df['Set'] = path.parent.parent.parent.parent.parent.name
        df['Categorie Montage'] = path.parent.parent.parent.parent.name
        df['Dossier Patient'] = path.parent.parent.parent.name
        df['Patient'] = path.parent.parent.name
        df['Session'] = path.parent.name
        df['Type'] = type
        r = re.search(r't\d{3}', str(path.name))
        if r:
            df['File N°'] = r.group()
        full_df = full_df.append(df, ignore_index=True)
    cols = full_df.columns.tolist()
    cols = cols[-6:] + cols[:-6]
    full_df = full_df[cols]
    return full_df

In [15]:
full_df_pan = load_and_concat_feats(path_pan,type='pan')

## creation d'un dataframe à partir des fichiers CSV de Lorenzo

In [4]:
df_pan = pd.read_csv("./pan.csv").drop('Unnamed: 0', axis=1)
df_swt = pd.read_csv("./swt.csv").drop('Unnamed: 0', axis=1)
df_xqrs = pd.read_csv("./xqrs.csv").drop('Unnamed: 0', axis=1)

In [145]:
### définition des clés

df_xqrs["cle_exam"]= df_xqrs['Set']+"_"+df_xqrs['Categorie Montage'].astype(str)
df_xqrs["cle_exam"]= df_xqrs["cle_exam"] +"_"+df_xqrs['Patient'].astype(str)
df_xqrs["cle_exam"]= df_xqrs["cle_exam"] +"_"+df_xqrs['Session'].astype(str)
df_xqrs["cle_exam"]= df_xqrs["cle_exam"] +"_"+df_xqrs['File N°'].astype(str)


df_swt["cle_exam"]= df_swt['Set']+"_"+df_swt['Categorie Montage'].astype(str)
df_swt["cle_exam"]= df_swt["cle_exam"] +"_"+df_swt['Patient'].astype(str)
df_swt["cle_exam"]= df_swt["cle_exam"] +"_"+df_swt['Session'].astype(str)
df_swt["cle_exam"]= df_swt["cle_exam"] +"_"+df_swt['File N°'].astype(str)


df_pan["cle_exam"]= df_pan['Set']+"_"+df_pan['Categorie Montage'].astype(str)
df_pan["cle_exam"]= df_pan["cle_exam"] +"_"+df_pan['Patient'].astype(str)
df_pan["cle_exam"]= df_pan["cle_exam"] +"_"+df_pan['Session'].astype(str)
df_pan["cle_exam"]= df_pan["cle_exam"] +"_"+df_pan['File N°'].astype(str)


In [97]:
list_features_non_linear = [
'csi',
'cvi',
'Modified_csi',
'sampen',
'sd1',
'sd2',
'ratio_sd2_sd1',
'label']

list_features_freq=[
'lf',
'hf',
'vlf',
'lf_hf_ratio']

list_features_temp=[
'mean_nni',
'sdnn',
'sdsd',
'nni_50',
'pnni_50',
'nni_20',
'pnni_20',
'rmssd',
'median_nni',
'range_nni',
'cvsd',
'cvnni',
'mean_hr',
'max_hr',
'min_hr',
'std_hr']

list_features = list_features_non_linear+list_features_freq+list_features_temp

In [148]:
df_result_xqrs=df_xqrs[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_xqrs[feat] = df_xqrs[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_xqrs[feat+"%"] = df_result_xqrs[feat]/df_result_xqrs["interval_index"]*100
    df_result_xqrs = df_result_xqrs.drop(feat,axis=1)
    
df_result_swt=df_swt[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_swt[feat] = df_swt[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_swt[feat+"%"] = df_result_swt[feat]/df_result_swt["interval_index"]*100
    df_result_swt = df_result_swt.drop(feat,axis=1)
    
df_result_pan=df_pan[['cle_exam','interval_index']].groupby("cle_exam").count()
for feat in list_features:
    df_result_pan[feat] = df_pan[['cle_exam',feat]].groupby("cle_exam").count()
    df_result_pan[feat+"%"] = df_result_pan[feat]/df_result_pan["interval_index"]*100
    df_result_pan = df_result_pan.drop(feat,axis=1)   

In [149]:
df_result_xqrs.sum().sort_values()/5080

lf_hf_ratio%      49.445828
lf%               49.632240
hf%               49.632240
vlf%              49.632240
interval_index    60.815945
csi%              66.852969
Modified_csi%     66.852969
ratio_sd2_sd1%    66.852969
cvi%              66.987081
sd1%              66.987081
sd2%              66.987081
sampen%           77.985971
cvnni%            84.843619
cvsd%             84.843619
std_hr%           84.843619
sdsd%             85.490437
rmssd%            85.490437
sdnn%             85.490437
median_nni%       86.573576
mean_nni%         86.573576
mean_hr%          86.573576
range_nni%        95.072562
max_hr%           95.072562
min_hr%           95.072562
nni_50%           95.837285
pnni_50%          95.837285
nni_20%           95.837285
pnni_20%          95.837285
label%            97.507047
dtype: float64

In [ ]:
df_result_xqrs_new = df_result_xqrs[df_result_xqrs.index.isin(keys)]

df_result_xqrs_new

In [150]:
df_result_xqrs

,interval_index,csi%,cvi%,Modified_csi%,sampen%,sd1%,sd2%,ratio_sd2_sd1%,label%,lf%,...,pnni_20%,rmssd%,median_nni%,range_nni%,cvsd%,cvnni%,mean_hr%,max_hr%,min_hr%,std_hr%
cle_exam,,,,,,,,,,,,,,,,,,,,,
dev_01_tcp_ar_10022_s001_2013_02_10_t000,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
dev_01_tcp_ar_10022_s001_2013_02_10_t001,127,87.401575,87.401575,87.401575,94.488189,87.401575,87.401575,87.401575,100.000000,74.015748,...,97.637795,92.125984,94.488189,97.637795,92.125984,92.125984,94.488189,97.637795,97.637795,92.125984
dev_01_tcp_ar_10022_s002_2013_02_10_t000,61,86.885246,86.885246,86.885246,88.524590,86.885246,86.885246,86.885246,98.360656,72.131148,...,98.360656,96.721311,96.721311,98.360656,96.721311,96.721311,96.721311,98.360656,98.360656,96.721311
dev_01_tcp_ar_10022_s002_2013_02_10_t001,61,88.524590,88.524590,88.524590,88.524590,88.524590,88.524590,88.524590,98.360656,73.770492,...,100.000000,96.721311,98.360656,100.000000,96.721311,96.721311,98.360656,100.000000,100.000000,96.721311
dev_01_tcp_ar_10022_s002_2013_02_10_t002,61,88.524590,88.524590,88.524590,88.524590,88.524590,88.524590,88.524590,98.360656,73.770492,...,100.000000,98.360656,100.000000,100.000000,98.360656,98.360656,100.000000,100.000000,100.000000,98.360656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
train_03_tcp_ar_a_9162_s003_2012_04_10_t006,61,1.639344,1.639344,1.639344,88.524590,1.639344,1.639344,1.639344,98.360656,4.918033,...,100.000000,22.950820,24.590164,96.721311,22.950820,22.950820,24.590164,96.721311,96.721311,22.950820
train_03_tcp_ar_a_9162_s003_2012_04_10_t007,17,0.000000,0.000000,0.000000,58.823529,0.000000,0.000000,0.000000,100.000000,0.000000,...,100.000000,29.411765,29.411765,94.117647,29.411765,29.411765,29.411765,94.117647,94.117647,29.411765
train_03_tcp_ar_a_9162_s003_2012_04_10_t008,61,54.098361,54.098361,54.098361,88.524590,54.098361,54.098361,54.098361,98.360656,36.065574,...,100.000000,73.770492,75.409836,100.000000,73.770492,73.770492,75.409836,100.000000,100.000000,73.770492


In [151]:
df_result_xqrs['interval_index_cat'] = df_result_xqrs["interval_index"].apply(lambda x: "0 to 1min" if (x<7) 
                                                                    else "1 to 2min30" if x<16 
                                                                    else ">2min30" )

df_result_xqrs['csi%_cat'] = df_result_xqrs["csi%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                else "40 to 60" if x<60
                                                else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_xqrs['mean_nni%_cat'] = df_result_xqrs["mean_nni%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                else "40 to 60" if x<60
                                                else "60 to 80" if x<80
                                                                    else "80 to 100" )

df_result_xqrs['lf%_cat'] = df_result_xqrs["lf%"].apply(lambda x: "0 to 20" if (x<20) 
                                                                    else "20 to 40" if x<40
                                                else "40 to 60" if x<60
                                                else "60 to 80" if x<80
                                                                    else "80 to 100" )

In [153]:
df_result_xqrs[['interval_index_cat','interval_index']].groupby('interval_index_cat').count()

,interval_index
interval_index_cat,
0 to 1min,163
1 to 2min30,208
>2min30,4709


In [154]:
df_result_xqrs[['csi%_cat','interval_index']].groupby('csi%_cat').agg(['count','sum'])

interval_index        
                   count     sum
csi%_cat                        
0 to 20              733   28511
20 to 40             239   10827
40 to 60             342   15497
60 to 80            1518   59779
80 to 100           2248  194331

In [155]:
df_result_xqrs[['mean_nni%_cat','interval_index']].groupby('mean_nni%_cat').agg(['count','sum'])

interval_index        
                       count     sum
mean_nni%_cat                       
0 to 20                  177   10424
20 to 40                 254   13536
40 to 60                 248   12911
60 to 80                 356   20026
80 to 100               4045  252048

In [156]:
df_result_xqrs[['lf%_cat','interval_index']].groupby('lf%_cat').agg(['count','sum'])

interval_index        
                   count     sum
lf%_cat                         
0 to 20             1189   43563
20 to 40             273   13830
40 to 60            1316   49819
60 to 80            1603  106983
80 to 100            699   94750